In [ ]:
import pandas as pd
import plotly.graph_objects as go
from enceladus.workflows import TestingPipeline

# Make Predictions

- Generate predictions on test data
- Find peaks and troughs to get SBP and DBP
- Calculate absolute error of BP estimations

## Error Grading System (British Hypertension Society Standard)
|         | <5 mmHg | <10 mmHg | <15 mmHg |
|   -:    | -:      | -:       | -:       |
| Grade A | 60%     | 85%      | 95%      |
| Grade B | 50%     | 75%      | 90%      |  
| Grade C | 40%     | 65%      | 85%      |

### MIMIC-III 50k (20230405) Test Results (serene-aardvark-11:v14)

|   Error  | Systolic | Diastolic |
|   -:     | -:       | -:        |
| < 15mmHg | 90.8%    | 97.3%     |
| < 10mmHg | 82.0%    | 94.4%     |
| <  5mmHg | 61.2%    | 83.4%     |

### MIMIC-III 200k (20230407) Test Results (glorious-fog-21:v5)

|   Error  | Systolic | Diastolic |
|   -:     | -:       | -:        |
| < 15mmHg | 82.6%    | 94.2%     |
| < 10mmHg | 71.2%    | 87.4%     |
| <  5mmHg | 48.1%    | 69.1%     |

### MIMIC-III 200k (20230408) Test Results (neat-voice-24:v11)

|   Error  | Systolic | Diastolic |
|   -:     | -:       | -:        |
| < 15mmHg | 86.3%    | 95.7%     |
| < 10mmHg | 76.8%    | 90.8%     |
| <  5mmHg | 55.7%    | 76.2%     |

In [ ]:
worker = TestingPipeline(
    data_dir='/home/camhpjohnson/data/mimic3-data-20230408/data/',
    model_dir='heartfelt/enceladus-fine-tuning/model-neat-voice-24:v11',
)

error, ppg, vpg, apg, abp, pred = worker.run()

In [ ]:
fig = go.Figure()

i = 3
fig.add_scatter(y=ppg[i, :]*80, name='ppg')
fig.add_scatter(y=abp[i, :], name='true')
fig.add_scatter(y=pred[i, :], name='pred')

In [ ]:
pd.options.plotting.backend = 'plotly'

fig = go.Figure()
fig.add_trace(go.Histogram(x=error['sbp_err'], name='SBP Error'))
fig.add_trace(go.Histogram(x=error['dbp_err'], name='DBP Error', opacity=0.5))

fig.update_layout(
        barmode='overlay',
        title={
            'text': 'Model Prediction Error Histogram',
            'font': {'size': 20},
        },
        xaxis={'title': 'Error (mmHg)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Number of Samples', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig.update_layout(
    plot_bgcolor='rgb(22,22,22)',
    paper_bgcolor='rgb(22,22,22)',
)
fig

In [ ]:
import plotly.graph_objects as go

i = 1

fig = go.Figure()

fig.add_scatter(
    y=abp[i, :],
    name='ABP (Ground Truth)',
)
fig.add_scatter(
    y=pred[i, :],
    name='Prediction',
)

fig.update_layout(
        title={
            'text': 'Prediction v Ground Truth',
            'font': {'size': 20},
        },
        xaxis={'title': 'Time (samples)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Amplitude (mmHg)', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig.update_layout(
    plot_bgcolor='rgb(22,22,22)',
    paper_bgcolor='rgb(22,22,22)',
)

fig